In [1]:
# (X_train,X_test),(y_train,y_test) = tensorflow.keras.datasets.cifar10.load_data()

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import os
from sys import getsizeof

# Load and preprocess CIFAR-10 dataset
def load_cifar10():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize to [0, 1]
    y_train, y_test = tf.keras.utils.to_categorical(y_train, 10), tf.keras.utils.to_categorical(y_test, 10)
    return (x_train, y_train), (x_test, y_test)

   


In [3]:

# Define ResNet-18 block
def resnet_block(inputs, filters, strides=1):
    x = layers.Conv2D(filters, kernel_size=3, strides=strides, padding='same', kernel_initializer='he_normal')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)

    if strides != 1 or inputs.shape[-1] != filters:
        inputs = layers.Conv2D(filters, kernel_size=1, strides=strides, padding='same', kernel_initializer='he_normal')(inputs)
    x = layers.Add()([x, inputs])
    x = layers.ReLU()(x)
    return x


In [4]:

# Define ResNet-18 model
def build_resnet18(input_shape=(32, 32, 3), num_classes=10):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same', kernel_initializer='he_normal')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
     # ResNet-18 has four stages with [2, 2, 2, 2] blocks
    filters = [64, 128, 256, 512]
    for i, num_blocks in enumerate([2, 2, 2, 2]):
        for j in range(num_blocks):
            strides = 2 if j == 0 and i != 0 else 1
            x = resnet_block(x, filters[i], strides)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    return model



In [10]:

# Training script
def train_resnet18():
    (x_train, y_train), (x_test, y_test) = load_cifar10()
    model = build_resnet18()
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

    # Print model summary
    model.summary()

    # Train the model
    model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))
    model.evaluate(x_test, y_test, verbose=2)
    model.save('Resnet_18_model.h5')
    

In [11]:

# Run the training process
if __name__ == "__main__":
    train_resnet18()
    

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 32, 32, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_20 (Conv2D)            │ (None, 16, 16, 64)        │           9,472 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_17        │ (None, 16, 16, 64)        │             256 │ conv2d_20[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ re_lu_17 (ReLU)               │ (None, 16, 16, 64)        │               0 │ batch_normalization_17[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_1               │ (None, 8, 8, 64)          │               0 │ re_lu_17[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_21 (Conv2D)            │ (None, 8, 8, 64)          │          36,928 │ max_pooling2d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_18        │ (None, 8, 8, 64)          │             256 │ conv2d_21[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ re_lu_18 (ReLU)               │ (None, 8, 8, 64)          │               0 │ batch_normalization_18[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_22 (Conv2D)            │ (None, 8, 8, 64)          │          36,928 │ re_lu_18[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_19        │ (None, 8, 8, 64)          │             256 │ conv2d_22[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_8 (Add)                   │ (None, 8, 8, 64)          │               0 │ batch_normalization_19[0]… │
│                               │                           │                 │ max_pooling2d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ re_lu_19 (ReLU)               │ (None, 8, 8, 64)          │               0 │ add_8[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_23 (Conv2D)            │ (None, 8, 8, 64)          │          36,928 │ re_lu_19[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_20        │ (None, 8, 8, 64)          │             256 │ conv2d_23[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 11,192,458 (42.70 MB)

 Trainable params: 11,184,650 (42.67 MB)

 Non-trainable params: 7,808 (30.50 KB)

Epoch 1/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 325s 802ms/step - accuracy: 0.4012 - loss: 2.0210 - val_accuracy: 0.3180 - val_loss: 2.3182
Epoch 2/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 312s 797ms/step - accuracy: 0.6242 - loss: 1.0589 - val_accuracy: 0.5557 - val_loss: 1.2160
Epoch 3/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 306s 783ms/step - accuracy: 0.6995 - loss: 0.8556 - val_accuracy: 0.5952 - val_loss: 1.1649
Epoch 4/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 326s 834ms/step - accuracy: 0.7504 - loss: 0.7164 - val_accuracy: 0.6311 - val_loss: 1.1342
Epoch 5/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 318s 813ms/step - accuracy: 0.7839 - loss: 0.6153 - val_accuracy: 0.6439 - val_loss: 1.1433
Epoch 6/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 329s 841ms/step - accuracy: 0.8216 - loss: 0.5065 - val_accuracy: 0.6133 - val_loss: 1.2808
Epoch 7/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 314s 802ms/step - accuracy: 0.8482 - loss: 0.4279 - val_accuracy: 0.6820 - val_loss: 1.0159
Epoch 8/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 307s 786ms/step - accuracy: 0.8777 -

In [12]:
model = tf.keras.models.load_model('Resnet_18_model.h5')